Totally fair question — this is *not* just a Jupyter trick. The exact same pattern works in:

* plain Python scripts (`.py`)
* IPython / Python REPL
* unit tests (pytest, unittest)
* even tiny one-off debug scripts you run from zsh

I’ll show you a few concrete ways to use it.

---

## 1. Using `debug_import` in a regular `.py` script

Say you have this helper in:

`telco_churn/utils/dev_tools.py`:

```python
from importlib import import_module, reload
from types import ModuleType
from typing import Iterable, Optional

def debug_import(module_path: str, attrs: Optional[Iterable[str]] = None) -> ModuleType:
    """
    Quick import debugger.

    Example:
        rep = debug_import("telco_churn.utils.reporting",
                           ["append_sec2", "log_section_completion"])
    """
    print(f"\n🔍 Importing module: {module_path}")
    mod = import_module(module_path)
    mod = reload(mod)

    print(f"📁 Module file: {getattr(mod, '__file__', '<no __file__>')}")

    if attrs:
        for name in attrs:
            has = hasattr(mod, name)
            print(f"   • has {name!r}? {has}")
    return mod
```

Now in a **normal script**, e.g. `debug_reporting.py`:

```python
# debug_reporting.py
from telco_churn.utils.dev_tools import debug_import

def main() -> None:
    rep = debug_import(
        "telco_churn.utils.reporting",
        attrs=["append_sec2", "log_section_completion"],
    )

    # You can now call the functions on `rep`
    # Example: rep.log_section_completion("demo", "OK", foo=1)

if __name__ == "__main__":
    main()
```

Run it from your project root (or anywhere with your `Level_3/src` on `PYTHONPATH`):

```bash
python debug_reporting.py
```

You’ll get the same info:

* which file was loaded
* whether `log_section_completion` exists

---

## 2. Using it in a plain Python REPL (no notebook)

From your terminal in the project:

```bash
python
```

Then:

```python
>>> from telco_churn.utils.dev_tools import debug_import
>>> rep = debug_import("telco_churn.utils.reporting",
...                    ["append_sec2", "log_section_completion"])
🔍 Importing module: telco_churn.utils.reporting
📁 Module file: /Users/b/DATA/PROJECTS/Telco/Level_3/src/telco_churn/utils/reporting.py
   • has 'append_sec2'? True
   • has 'log_section_completion'? True

>>> rep.log_section_completion("2.1.5", "OK", checked=10, mismatched=0)
✅ [2.1.5] | status=OK | checked=10 | mismatched=0
```

Same move in IPython:

```bash
ipython
```

Then same `debug_import(...)` call.

---

## 3. Using it inside tests (pytest)

This is actually a *great* place to use it when something is off.

```python
# tests/test_reporting_imports.py

from telco_churn.utils.dev_tools import debug_import

def test_reporting_has_log_section():
    rep = debug_import(
        "telco_churn.utils.reporting",
        attrs=["append_sec2", "log_section_completion"],
    )
    assert hasattr(rep, "append_sec2")
    assert hasattr(rep, "log_section_completion")
```

Run:

```bash
pytest -q
```

If you mess up the file path, or forget to save, the test will immediately tell you.

---

## 4. A tiny CLI “import doctor” you can run from anywhere

If you want something you can call with *any* module + attributes:

```python
# tools/import_doctor.py

import argparse
from telco_churn.utils.dev_tools import debug_import

def main() -> None:
    parser = argparse.ArgumentParser()
    parser.add_argument("module", help="Module path, e.g. telco_churn.utils.reporting")
    parser.add_argument(
        "--attrs", nargs="*", default=[],
        help="Attribute names to check for on the module",
    )
    args = parser.parse_args()

    debug_import(args.module, attrs=args.attrs or None)

if __name__ == "__main__":
    main()
```

Usage:

```bash
python tools/import_doctor.py telco_churn.utils.reporting --attrs append_sec2 log_section_completion
```

That’s your “import debugger” as a script, no notebook required.

---

## 5. Mental model for when to reach for this tool

Reach for `debug_import` whenever:

* You edited a module, but your script still behaves like the old version.
* You added a function, but `from ... import ...` says `ImportError`.
* You’re not sure *which copy* of the module is being imported.

Your workflow becomes:

1. Call `debug_import("some.module", ["some_attr"])`
2. Fix file / path / typos until it says:

   * correct `__file__`
   * `has 'some_attr'? True`
3. Once good, swap back to normal imports.

---

If you want, I can:

* Drop a ready-to-save `dev_tools.py` plus `import_doctor.py` for Telco/Level_3 with paths matched to your layout, so you just paste them in and go.
